```
bryan@ichabod mac_bucket % for file in *_raster_log.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/raster_log_raw/
done
```


In [ ]:
df = spark.read.format("json").load("/Volumes/geodata/petra/raster_log_raw/")

display(df)

In [ ]:
from common.transforms import generate_hash
import pyspark.sql.functions as F

df_flat = df.select(
    F.col("repo_id"),
    F.col("uwi.uwi").alias("uwi"),
    F.col("uwi.wsn").alias("wsn"),
    F.col("logimage.calibfilename").alias("calibfilename"),
    F.col("logimage.ign").alias("ign"),
    F.col("logimage.imagefilename").alias("imagefilename"),
    F.col("logimgrp.desc").alias("description"),
    F.col("logimgrp.groupname").alias("groupname"),
    F.col("logimgrp.path").alias("path"),
)

df_raster_log = df_flat


# add id hash
id_columns = ["repo_id", "uwi"]
df_raster_log = generate_hash(df_raster_log, "id", "raster_log", *id_columns)


display(df_raster_log)

In [ ]:
from common.transforms import upsert_dataframe_to_table

result = upsert_dataframe_to_table(df_raster_log, "geodata.petra.raster_log_bronze")
display(result)